In [1]:
from nplm import PartialLabelModel
import numpy as np
import torch
import joblib

%load_ext autoreload
%autoreload 2

In [2]:
############################
# 1. Load Detectors Votes
votes = np.load('votes/awa2_unseen_votes.npy')

# votes = np.load('votes/unseen_votes.npy')


############################
# 2. Load Class-Attribute CFG (Specifying attribute as partial label) as fid2clusters
fid2clusters = np.load('data/awa2/fid2clusters.npy', allow_pickle=True).item()

In [3]:
lm_cfg = {'lr': 1,
 'epoch': 300,
 'seed': 0,
 'batch_size': 8192,
 'momentum': 0.9,
 'step_schedule': 'p',
 'step_multiplier': 0.1}

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
labelmodel = PartialLabelModel(num_classes=10,
                            fid2clusters=fid2clusters,
                            opt_cfg=lm_cfg,
                            device=device, verbose=True)

labelmodel.optimize(votes)

epoch   2:   0%|          | 0/1 [00:00<?, ?it/s]

Setup:  0.13570833206176758


epoch  108: 100%|██████████| 1/1 [00:00<00:00,  9.50it/s, Epoch Loss: =19.5]


epoch  214: 100%|██████████| 1/1 [00:00<00:00,  9.56it/s, Epoch Loss: =19.4]


epoch  274: 100%|██████████| 1/1 [00:00<00:00,  9.81it/s, Epoch Loss: =19.4]

1e-7 Criterion Reached: Epoch  273


In [4]:
label_estim = labelmodel.weak_label(votes)

Setup:  0.12729907035827637
Parallel Estimation:  0.02508997917175293


In [5]:
from random import shuffle
import random
from torch.utils.data import DataLoader
from end import ResNetFeaturesLC
from end.Tasks import EndTask
from sklearn.preprocessing import normalize

def awapipeline_gzsl(soft_label, seed=-1):

    if seed != -2:
        seeds = [500, 600, 700, 800, 900]
        rs = seeds[seed]
        random_seed = rs
        pt_seed = rs
        np_seed = pt_seed
        random.seed(random_seed)
        torch.manual_seed(pt_seed)
        np.random.seed(np_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(pt_seed)

    unseen_features = joblib.load('data/awa2/unseen_features.jkl')
    unseen_inst, _ = list(zip(*unseen_features[0]))
    unseen_real_labels = list(unseen_features[1].transpose()[0]- 1)
    unseen_inst = np.array(list(unseen_inst))
    unseen_soft_labels = np.array(list(soft_label))
    num_unseen = len(unseen_inst)
    test_data = list(zip(unseen_inst, unseen_real_labels))
    
    unseen_soft_labels_augmented = np.zeros([num_unseen, 50])
    unseen_soft_labels_augmented[:, np.array(np.unique(unseen_real_labels))] = unseen_soft_labels

    seen_all = joblib.load('data/awa2/seen_features.jkl')
    seen_labels = list(seen_all[1].transpose()[0])
    seen_inst, _ = list(zip(*seen_all[0]))
    translated_seenlabels = np.array(seen_labels) - 1
    seen_test_index = np.load('data/awa2/awa2_seen_test_idx.npy')

    seen_test = []
    seen_train_inst = []
    seen_train_labels = []
    for idx in range(len(seen_labels)):
        if idx in seen_test_index:
            seen_test.append((seen_inst[idx], translated_seenlabels[idx]))
        else:
            seen_train_inst.append(seen_inst[idx])
            seen_train_labels.append(translated_seenlabels[idx])

    one_hot_translator = np.eye(50)
    seen_train_soft_labels = one_hot_translator[np.array(seen_train_labels)]
    seen_train_soft_labels[:, np.array(np.unique(unseen_real_labels))] *= 0
    train_inst = np.vstack([unseen_inst, torch.DoubleTensor(seen_train_inst)])

    train_soft_labels = np.vstack([unseen_soft_labels_augmented, seen_train_soft_labels])
    train_inst = normalize(train_inst)
    train = list(zip(train_inst, train_soft_labels))
    test = test_data + seen_test
    valtest_inst, valtest_labels = list(zip(*test))
    valtest_inst = normalize(valtest_inst)
    test = list(zip(valtest_inst, valtest_labels))

    train = DataLoader(train, shuffle=True, batch_size=16)
    valtest = DataLoader(test, shuffle=True, batch_size=32)

    model_const = ResNetFeaturesLC
    task = EndTask('AwA2-GZSL', 50, model_const)
    task.train(train, valtest, valtest, gzsl=True, std=True)

In [6]:
awapipeline_gzsl(label_estim)

Epoch 1: MCA: Seen:  0.9074532802853579 Unseen:  0.7081024438294368 H:  0.7954784546762466
Epoch 2: MCA: Seen:  0.9122864242927191 Unseen:  0.7114790319115504 H:  0.7994660306410415
Epoch 3: MCA: Seen:  0.9186510190927141 Unseen:  0.7087644628996713 H:  0.8001732849958592
Epoch 4: MCA: Seen:  0.9134685206386678 Unseen:  0.6957507735498317 H:  0.7898816924989309
Epoch 5: MCA: Seen:  0.9141440963331583 Unseen:  0.68916933043468 H:  0.7858726363450619
Epoch 6: MCA: Seen:  0.9151265883784367 Unseen:  0.7072816348402597 H:  0.7978907160985738
Epoch 7: MCA: Seen:  0.9192250156411355 Unseen:  0.7061109373342422 H:  0.7986962156681937
Epoch 8: MCA: Seen:  0.9191177636436901 Unseen:  0.716722327330686 H:  0.8053992884564897
Epoch 9: MCA: Seen:  0.916924404146268 Unseen:  0.7259115208240103 H:  0.8103134081469624
Epoch 10: MCA: Seen:  0.9185398572592284 Unseen:  0.70498253709494 H:  0.7977155858709665
Epoch 11: MCA: Seen:  0.9144147999585304 Unseen:  0.7192012023099826 H:  0.8051441986696719
